In [1]:
# mission_to_mars.ipynb
# Eric Staveley  MWsa

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
from splinter import Browser
import time

In [3]:
#Mac user [prob TA/Instr  :-) ]
# Choose the executable path to driver 
#executable_path = {'executable_path': '/usr/local/bin/chromedriver.exe'}
#browser = Browser('chrome', **executable_path, headless=False)

In [4]:
def init_browser():
    #windows (me)   Make sure the chromedriver is in the jupy nb file location
    executable_path = {"executable_path": 'chromedriver.exe'}
    return Browser("chrome", **executable_path, headless=False)

# NASA Mars News Scraping

In [5]:
# Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. 
# Assign the text to variables that you can reference later.

In [9]:
browser = init_browser()

In [10]:
# NASA mars url
url_mars = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

In [11]:
#confirm the site
browser.visit(url_mars)

In [12]:
#define html object
html_mars = browser.html

In [13]:
#use BeautifulSoup to extract data from html
soup_mars = BeautifulSoup(html_mars, 'lxml')

In [14]:
#get latest news title and show it
news_title = soup_mars.find('div', class_='content_title').text
news_title

"Curiosity Tastes First Sample in 'Clay-Bearing Unit'"

In [15]:
#find unique tag and get latest news paragraph and show it
news_paragraph = soup_mars.find('div', class_='article_teaser_body').text
news_paragraph

'This new region on Mars might reveal more about the role of water on Mount Sharp.'

# JPL Scraping

In [16]:
# Visit the url for JPL Featured Space Image
# Use splinter to navigate the site and find the image url for the current 
# Featured Mars Image and assign the url string to a variable called featured_image_url.
# Make sure to find the image url to the full size .jpg image.
# Make sure to save a complete url string for this image.

In [17]:
#Find mars images from JPL url
url_jpl= "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"

#define base jpl url
base_url = 'https://www.jpl.nasa.gov/'

#confirm the site
browser.visit(url_jpl)

#use is_text_present and click_link_by_partial_text to help define browser
browser.is_text_present('FULL IMAGE')
browser.click_link_by_partial_text('FULL IMAGE')

In [18]:
#define html object
html_jpl = browser.html

In [19]:
#use BeautifulSoup to extract data from html
soup_jpl = BeautifulSoup(html_jpl, 'lxml')

In [20]:
#find unique tag to locate data of interest
image_location = soup_jpl.find('article')['style'].replace('background-image: url(', '').replace(');', '')[1:-1]

In [21]:
featured_image_url = base_url + image_location
featured_image_url    #  the full image url

'https://www.jpl.nasa.gov//spaceimages/images/wallpaper/PIA14884-1920x1200.jpg'

# Scrape Mars Weather twitter site

In [22]:
# Visit the Mars Weather twitter account and scrape the latest Mars weather tweet from the page. 
# Save the tweet text for the weather report as a variable called mars_weather.

In [23]:
#scrape mars weather from their twitter account
url_mars_weather = 'https://twitter.com/marswxreport?lang=en'
#verify site
browser.visit(url_mars_weather)

In [24]:
#define html object
html_mars_weather = browser.html

In [25]:
#use BeautifulSoup to extract data from html
soup_mars_weather = BeautifulSoup(html_mars_weather, 'lxml')

In [27]:
#find unique tag to locate data of interest, weather info is just below div class js-tweet-text-container
weather_info = soup_mars_weather.find_all('div', class_='js-tweet-text-container')
weather_info

[<div class="js-tweet-text-container">
 <p class="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text" data-aria-label-part="0" lang="en">InSight sol 135 (2019-04-13) low -96.5ºC (-141.8ºF) high -16.6ºC (2.2ºF)
 winds from the SW at 4.2 m/s (9.4 mph) gusting to 11.3 m/s (25.3 mph)
 pressure at 7.30 hPa<a class="twitter-timeline-link u-hidden" data-pre-embedded="true" dir="ltr" href="https://t.co/bRsLlzn4M0">pic.twitter.com/bRsLlzn4M0</a></p>
 </div>, <div class="js-tweet-text-container">
 <p class="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text" data-aria-label-part="0" lang="en">InSight sol 134 (2019-04-12) low -98.4ºC (-145.1ºF) high -18.2ºC (-0.8ºF)
 pressure at 7.30 hPa<a class="twitter-timeline-link u-hidden" data-pre-embedded="true" dir="ltr" href="https://t.co/1DE7KRn8xy">pic.twitter.com/1DE7KRn8xy</a></p>
 </div>, <div class="js-tweet-text-container">
 <p class="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text" data-aria-label-part="0" lang="e

In [28]:
#some tweets may not be about the weather....so we will loop thru 
#until we find 'sol' 'high' 'low' and 'pressure' strings
for item in weather_info:
    tweet_weather = item.find('p').text
    if 'sol' and 'low' and 'high' and 'pressure' in tweet_weather:
        break   # can stop looking

tweet_weather

'InSight sol 135 (2019-04-13) low -96.5ºC (-141.8ºF) high -16.6ºC (2.2ºF)\nwinds from the SW at 4.2 m/s (9.4 mph) gusting to 11.3 m/s (25.3 mph)\npressure at 7.30 hPapic.twitter.com/bRsLlzn4M0'

In [29]:
# strip new line char and trim off pic and beyond, and  and save as mars_weather
temp = tweet_weather.replace('\n',' ')
new_temp = temp[:temp.rfind('pic')]
mars_weather = new_temp
mars_weather

'InSight sol 135 (2019-04-13) low -96.5ºC (-141.8ºF) high -16.6ºC (2.2ºF) winds from the SW at 4.2 m/s (9.4 mph) gusting to 11.3 m/s (25.3 mph) pressure at 7.30 hPa'

# Scrape Mars facts url

In [30]:
# Visit the Mars Facts webpage and use Pandas to scrape the table containing facts 
# about the planet including Diameter, Mass, etc.

# Use Pandas to convert the data to a HTML table string.

In [31]:
#scrape mars facts page using pandas
url_facts = 'https://space-facts.com/mars/'

In [32]:
#parse the url with read_html
mars_tables = pd.read_html(url_facts)
mars_tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [33]:
#make the mars fact df out of the mars_table list
df = mars_tables[0]
df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [34]:
df = mars_tables[0]
df.columns = ['Mars item', 'Value']

In [35]:
df

,Mars item,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [36]:
html_tbl_str = df.to_html()
html_tbl_str

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars item</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium B

In [38]:
# stript unwanted newlines to clean up the table string
html_str_stripped = html_tbl_str.replace('\n',"")

In [39]:
#for fun, save the table to a file to see what it looks like
df.to_html('mars_facts_raw.html')

# Get images of Mars hemispheres from USGS Astrogeology url

In [40]:
# Visit the USGS Astrogeology site to obtain high resolution images for each of Mar's hemispheres.
# You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.
# Save both the image url string for the full resolution hemisphere image, and 
# the Hemisphere title containing the hemisphere name. 
# Use a Python dictionary to store the data using the keys img_url and title.

# Append the dictionary with the image url string and the hemisphere title to a list. 
# This list will contain one dictionary for each hemisphere.

In [41]:
#define url
usgs_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
#verify
browser.visit(usgs_url)
#define base url
hemis_base_url = 'https://astrogeology.usgs.gov'

In [42]:
#define html object
html_hemis = browser.html

In [43]:
#use BeautifulSoup to extract data from html
soup_jpl = BeautifulSoup(html_hemis, 'lxml')

In [44]:
# find unique tag to locate data of interest 
cls_item = soup_jpl.find_all('div', class_='item')

In [45]:
#make/clear empty list to hold our image list
hemis_img_list = []

In [46]:
for item in cls_item:
    
    #hemisphere titles found in h3 tag
    img_title = item.find('h3').text
    
    #find sub url of image, found in cls itemLink product-item
    sub_img_url = item.find('a', class_='itemLink product-item')['href']
    
    #need to click each of the links to the hemispheres in order to 
    browser.visit(hemis_base_url + sub_img_url)
    #put in any delayes here? i.e :    time.sleep(2)
    time.sleep(2)
    
    #now define html object
    img_html = browser.html
    
    #use BeautifulSoup to extract data from html
    soup_img = BeautifulSoup(img_html, 'lxml')
    #the specific image path ending
    endpoint = soup_img.find('img', class_='wide-image')['src']
    #print(endpoint)
    
    #construct the image url to the full resolution image
    img_url = hemis_base_url + endpoint
    
    #append to our list we created
    hemis_img_list.append({"title" : img_title, "img_url" : img_url})
    
    

In [47]:
hemis_img_list

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]

# Utility tasks (not to be used in further py activites)

In [ ]:
#convert the jupy bn file to a python script
!jupyter nbconvert --to python mission_to_mars.ipynb

In [ ]:
#make a scrape_mars.py script file out of the conversion
!copy mission_to_mars.py scrape_mars.py